In [1]:
# Imports
from collections import defaultdict, Counter, deque
from functools import lru_cache, reduce, cmp_to_key
import itertools
import json
import numpy as np
import numpy.typing as npt
import re
from sortedcontainers import SortedList
from typing import Callable, TypeVar, Union, Optional


T = TypeVar('T')

# Helper functions

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

def split_csv_row(row: str) -> list[int]:
  return [int(x) for x in row.split(',')]

def split_int_row(row: str) -> list[int]:
  return [int(x) for x in row]

# Day 1

In [19]:
def parse_day1(data):
  elves = SortedList()
  current = 0
  for food in data:
    if not food:
      elves.add(current)
      current = 0
    else:
      current += int(food)
  return elves

data1 = parse_day1(data(1))

In [21]:
data1[-1]

71780

In [23]:
sum(data1[-3:])

212489

# Day 2

In [13]:
def parse2(data):
  result = Counter()
  for round in data:
    result[round] += 1
  return result
data2 = parse2(data(2))

In [11]:
score = {
  'A X': 3 + 1, # Rock Rock
  'A Y': 6 + 2, # Rock Paper
  'A Z': 0 + 3, # Rock Scissor
  'B X': 0 + 1, # Paper Rock
  'B Y': 3 + 2, # Paper Paper
  'B Z': 6 + 3, # Paper Scissor
  'C X': 6 + 1, # Scissor Rock
  'C Y': 0 + 2, # Scissor Paper
  'C Z': 3 + 3, # Scissor Scissor
}
sum([score[x]*data2[x] for x in data2])

13924

In [12]:
score = {
  'A X': 0 + 3, # Rock Lose Sciss
  'A Y': 3 + 1, # Rock Draw Rock
  'A Z': 6 + 2, # Rock Win Paper
  'B X': 0 + 1, # Paper Lose Rock
  'B Y': 3 + 2, # Paper Draw Paper
  'B Z': 6 + 3, # Paper Win Sciss
  'C X': 0 + 2, # Scissor Lose Paper
  'C Y': 3 + 3, # Scissor Draw Sciss
  'C Z': 6 + 1, # Scissor Win Rock
}
sum([score[x]*data2[x] for x in data2])

13448

# Day 3

In [29]:
data3 = data(3)

def item_priority(item):
  if ord(item) > 96:
    return ord(item) - 96
  return ord(item) - 38

In [31]:
def rucksack_items(rucksack):
  length = len(rucksack)//2
  return rucksack[:length], rucksack[length:]
  
def day3(data):
  sum = 0
  for x in data:
    a, b = rucksack_items(x)
    sum += item_priority(list(set(a).intersection(set(b)))[0])
  return sum

day3(data3)

7727

In [36]:
def day3_2(data):
  sum = 0
  for i in range(len(data)//3):
    a, b, c = data[i*3:i*3+3]
    common = set(a).intersection(set(b)).intersection(set(c))
    sum += item_priority(list(common)[0])
  return sum

day3_2(data3)

2609

# Day 4

In [43]:
data4 = data(4)

In [41]:
def has_envelop(a, b):
  return a[0] <= b[0] and a[1] >= b[1]

def day4(data, f):
  count = 0
  for i in data:
    a, b = [[int(x) for x in elf.split('-')] for elf in i.split(',')]
    count += f(a, b) or f(b, a)
  return count

day4(data4, has_envelop)

2

In [44]:
def has_overlap(a, b):
  return (a[0] <= b[0] and a[1] >= b[0]) or (a[0] >= b[0] and a[0] <= b[1])

day4(data4, has_overlap)

891

# Day 5

In [2]:
def make_move(positions, count, start, end):
  if count == 0:
    return
  item = positions[start].popleft()
  positions[end].appendleft(item)
  return make_move(positions, count-1, start, end)

def read_top(positions):
  return ''.join([positions[i][0] for i in range(len(positions))])

def day5(make_move):
  def parse5():
    with open('./data/day5.txt') as f:
      lines = f.readlines()
    positions = defaultdict(lambda: deque())
    line_length = len(lines[0])
    for n, line in enumerate(lines):
      for i in range(line_length//4):
        item = line[i*4:i*4+4].strip()
        if item == '1':
          return positions, lines[n+2:]
        if item:
          positions[i].append(re.match('\[([A-Z])\]', item)[1])
  positions, moves = parse5()
  for turn in moves:
    result = re.match('move (\d+) from (\d) to (\d)', turn)
    make_move(positions, int(result[1]), int(result[2])-1, int(result[3])-1)
  return read_top(positions)

day5(make_move)

'RFFFWBPNS'

In [3]:
def make_move_9001(positions, count, start, end):
  stack = deque()
  for _ in range(count):
    stack.appendleft(positions[start].popleft())
  positions[end].extendleft(stack)

day5(make_move_9001)

'CQQBBJFCS'

# Day 6

In [5]:
data6 = data(6)[0]

In [6]:
def day6(n):
  for i in range(len(data6)-n+1):
    if len(set(data6[i:i+n])) == n:
      return i + n

day6(4), day6(14)

(1262, 3444)

# Day 7

In [6]:
class Dir:
  def __init__(self, parent=None):
    self.parent = parent
    self.files = 0
    self.children = {}

def ls(item, cwd):
  if item.startswith('dir'):
    cwd.children[item[4:]] = Dir(cwd)
  else:
    cwd.files += int(item.split(' ')[0]) 

def cd(cmd, cwd, root):
  if cmd.startswith('cd'):
    cwd = root if cmd == 'cd /' else cwd.parent if '..' in cmd else cwd.children[cmd.split(' ')[1]]
  return cwd

def parse7(data):
  root = Dir()
  cwd = root
  for line in data:
    if line[0] == '$':
      cwd = cd(line[2:], cwd, root)
    else:
      ls(line, cwd)
  return root

data7 = []
def calculate_total_sizes(root):
  total = root.files
  for child in root.children:
    total += calculate_total_sizes(root.children[child])
  data7.append(total)
  return total
total = calculate_total_sizes(parse7(data(7)))

In [7]:
sum([x for x in data7 if x <= 100000])

1206825

In [8]:
target = total - (70000000 - 30000000)
next(filter(lambda x: x >= target, sorted(data7)))

9608311

# Day 8

In [289]:
data8 = np.array(data(8, lambda x: [int(n) for n in x]))
y, x = data8.shape

In [290]:
maskn, masks = [np.full([1, x], True)[0]], []
for i in range(x-1):
  forward = np.amax(data8[:i+1], 0)
  maskn.append(data8[i+1] > forward)
  backward = np.amax(data8[i+1:], 0)
  masks.append(data8[i] > backward)
masks.append(np.full([1, x], True)[0])

maskw, maske =  [np.full([y, 1], True)[:, 0]], []
for i in range(y-1):
  forward = np.amax(data8[:, :i+1], 1)
  maskw.append(data8[:, i+1] > forward)
  backward = np.amax(data8[:, i+1:], 1)
  maske.append(data8[:, i] > backward)
maske.append(np.full([y, 1], True)[:, 0])

mask = np.vstack(maskn) | np.vstack(masks) | np.vstack(maskw).transpose() | np.vstack(maske).transpose()
len(data8[np.where(mask)])

1816

In [288]:
def directional_score(geni, genj, axis=0):
  score = [np.zeros([1, x])[0]]
  for i in geni:
    current = data8[i] if axis == 0 else data8[:, i]
    visible = np.full([1, x], True)[0]
    result = np.zeros([1, x])[0]
    for j in genj(i):
      candidate = data8[j] if axis == 0 else data8[:, j]
      result += np.ones([1, x])[0] * visible
      visible = (current > candidate) & visible
    score.append(result)
  return np.vstack(score)

north = directional_score(range(1, x), lambda i: range(i-1, -1, -1))
south = np.flip(directional_score(range(x-2, -1, -1), lambda i: range(i+1, x)), 0)
west = directional_score(range(1, x), lambda i: range(i-1, -1, -1), 1).transpose()
east = np.flip(directional_score(range(x-2, -1, -1), lambda i: range(i+1, x), 1).transpose(), 1)
int(np.max(north * south * east * west))

383520

# Day 9

In [3]:
data9 = data(9, lambda x: x.split(' '))

In [6]:
def day9(data, knots=2):
  visited = set()
  state = [(0,0)] * knots

  def move_head(x, y, direction):
    match direction:
      case 'R':
        x += 1
      case 'L':
        x -= 1
      case 'U':
        y += 1
      case 'D':
        y -= 1
    return (x, y)

  def move_tail(headx, heady, tailx, taily):
    if abs(headx - tailx) > 1 and abs(heady - taily) > 1:
      return (headx + (1 if tailx > headx else -1), heady + (1 if taily > heady else -1))
    if abs(headx - tailx) > 1:
      return (headx + (1 if tailx > headx else -1), heady)
    if abs(heady - taily) > 1:
      return (headx, heady + (1 if taily > heady else -1))
    return (tailx, taily)

  def make_move(current, direction, count):
    for _ in range(count):
      current[0] = move_head(*current[0], direction)
      for i in range(1, len(current)):
        current[i] = move_tail(*current[i-1], *current[i])
      visited.add(current[-1])
    return current

  for move in data:
    state = make_move(state, move[0], int(move[1]))

  return len(visited)

day9(data9), day9(data9, 10)

(6181, 2386)

# Day 10

In [10]:
def parse10(data):
  q, i, state, op = deque([0]), 0, 1, 0

  def tock(i, state, op):
    if not op:
      if q:
        op = q.popleft()
    else:
      state += op
      op = 0
    i += 1
    return i, state, op

  for command in data:
    q.append(0 if len(command) == 1 else int(command[1]))
    i, state, op = tock(i, state, op)
    yield state
  while q:
    i, state, op = tock(i, state, op)
    yield state
  i, state, op = tock(i, state, op)
  yield state

data10 = lambda: parse10((data(10, lambda x: x.split(' '))))

In [11]:
def day10_1(data):
  signal = 0
  targets = [20, 60, 100, 140, 180, 220]
  for i, v in enumerate(data):
    if i+1 in targets:
      signal += (i+1) * v
  return signal
day10_1(data10())

14420

In [23]:
def day10_2(data):
  image = ''
  for i, v in enumerate(data):
    position, sprite = i+1, v
    if position%40 in range(sprite, sprite+3):
      image += '#'
    else:
      image += ' '
    if not position%40:
      image += '\n'
  return image
print(day10_2(data10()))

###   ##  #    ###  ###  ####  ##  #  # 
#  # #  # #    #  # #  #    # #  # #  ##
#  # #    #    #  # ###    #  #  # #  # 
###  # ## #    ###  #  #  #   #### #  ##
# #  #  # #    # #  #  # #    #  # #  ##
#  #  ### #### #  # ###  #### #  #  ##  
  


# Day 11

In [201]:
# Hardcoded
example = [
  lambda x: x*19,
  lambda x: x+6,
  lambda x: x*x,
  lambda x: x+3
]
actual = [ 
  lambda x: x*13,
  lambda x: x+2,
  lambda x: x+6,
  lambda x: x*x,
  lambda x: x+3,
  lambda x: x*7,
  lambda x: x+4,
  lambda x: x+7
]
data11 = (data(11), actual)

In [205]:
def parse11(data, ops):
  monkeys, divisor = [], 1

  class Monkey:
    def __init__(self, items, op, n, a, b):
      self.items = items
      self.operation = op
      self.n = n
      self.a = a
      self.b = b
      self.count = 0

    def run(self, flag=False):
      t, f = monkeys[self.a], monkeys[self.b]
      for old in self.items:
        new = (self.operation(old) // (1 if flag else 3)) % divisor
        (f if new%self.n else t).items.append(new)
        self.count += 1
      self.items = []

    def __repr__(self):
      return f'{self.count}, {self.items}'

  monkey = {}
  for line in data:
    if not line:
      monkeys.append(Monkey(**monkey))
    if m := re.match('Starting items: ([\d+, ]+)', line):
      monkey['items'] = [int(x) for x in m[1].split(', ')]
    elif m:= re.match('Operation: new = old (.) (.+)', line):
      monkey['op'] = ops[len(monkeys)]
    elif m:= re.match('Test: divisible by (\d+)', line):
      monkey['n'] = int(m[1])
      divisor *= monkey['n']
    elif m:= re.match('If true: throw to monkey (\d)', line):
      monkey['a'] = int(m[1])
    elif m:= re.match('If false: throw to monkey (\d)', line):
      monkey['b'] = int(m[1])
  monkeys.append(Monkey(**monkey))
  return monkeys

def day11(data, rounds, flag=False):
  monkeys = parse11(*data)
  for _ in range(rounds):
    for monkey in monkeys:
      monkey.run(flag)
  activity = sorted(monkeys, key=lambda x: x.count, reverse=True)
  return activity[0].count * activity[1].count

day11(data11, 20)

121450

In [204]:
day11(data11, 10000, True)

28244037010

# Day 12

In [41]:
def remap12(c):
  return 0 if c == 'S' else (25 if c == 'E' else ord(c) - ord('a'))

def parse12(data):
  start, end, result = 0, 0, []
  for i, line in enumerate(data):
    if 'S' in line:
      start = (line.find('S'), i)
    if 'E' in line:
      end = (line.find('E'), i)
    result.append([remap12(c) for c in line])

  x_m, y_m = len(result[0]), len(result)
  def find_routes(current, i):
    x, y = current
    options = filter(lambda x: x[0] >= 0 and x[1] >= 0 and x[0] < x_m and x[1] < y_m, [(x, y+1), (x, y-1), (x+1, y), (x-1, y)])
    target = result[y][x] -1
    return [(coord, i+1) for coord in filter(lambda x: result[x[1]][x[0]] >= target, options)]

  return start, end, find_routes

data12 = data(12)
start, end, find_routes = parse12(data12)

def dijkstra(start, end, route_finder, cache={}):
  visited, q = set(start), deque(route_finder(start, 0))
  while q:
    current, i = q.popleft()
    if current == end:
      return i
    for option, j in route_finder(current, i):
      if option not in visited:
        q.append((option, j))
        cache[option] = j
      visited.add(option)
  return np.Inf

dijkstra(end, start, find_routes)

339

In [55]:
def find_starts(data):
  starts = []
  for y, line in enumerate(data):
    for x, c in enumerate(line):
      if c == 'a' or c == 'S':
        starts.append((x, y))
  return sorted(starts, key=lambda x: abs(x[0]-end[0]) + abs(x[1]-end[1]), reverse=True)

cache = {}
starts = find_starts(data12)
for start in starts:
  result = dijkstra(end, start, find_routes, cache)
  if result == np.Inf:
    break
min([cache.get(start, np.Inf) for start in starts])

332

# Day 13

In [4]:
def parse13(data):
  current, pairs = None, []
  for line in data:
    if not line:
      continue
    if not isinstance(current, list):
      current = json.loads(line)
    else:
      pairs.append((current, json.loads(line)))
      current = None
  return pairs

data13 = parse13(data(13))

In [6]:
def compare_int(a, b):
  if a == b:
    return 0
  return -1 if a < b else 1

def make_list(item):
  if isinstance(item, list):
    return item
  return [item]

def compare_list(a, b):
  for i in range(min(len(a), len(b))):
    if result := compare(a[i], b[i]):
      return result
  if len(a) == len(b):
    return 0
  return -1 if len(a) < len(b) else 1

def compare(a, b):
  if isinstance(a, list) or isinstance(b, list):
    return compare_list(make_list(a), make_list(b))
  return compare_int(a, b)

def day13(data):
  sum = 0
  for i, pair in enumerate(data):
    if compare(*pair) == -1:
      sum += i+1
  return sum

day13(data13)

5366

In [7]:
def day13_2(data):
  dividers = ([[2]], [[6]])
  data = reduce(lambda x, y: x+list(y), data13 + [dividers], [])
  result = sorted(data, key=cmp_to_key(compare))
  indices = [result.index(x)+1 for x in dividers]
  return indices[0]*indices[1]

day13_2(data13)

23391

# Day 14

In [53]:
def parse14(data):
  results, bottom = set(), 0
  for line in data:
    coordinates = [[int(x) for x in coord.split(',')] for coord in line.split(' -> ')]
    x1, y1 = coordinates[0]
    for (x2, y2) in coordinates[1:]:
      for i in range(min(x1, x2), max(x1,x2)+1):
        for j in range(min(y1, y2), max(y1,y2)+1):
          results.add((i, j))
      bottom = max(bottom, j)
      x1, y1 = x2, y2
  return results, bottom
  
data14 = parse14(data(14))

In [54]:
def day14(sand, bottom, flag=False):  
  sand, start, initial, path = sand.copy(), (500, 0), len(sand), deque()
  while start not in sand:
    x, y = path.pop() if path else start
    while (x, y) not in sand:
      for candidate in ((x, y+1), (x-1, y+1), (x+1, y+1)):
        if candidate not in sand:
          path.append(candidate)
          x, y = candidate
          break
      if candidate in sand or candidate[1] > bottom:
        sand.add((x, y))
    if not flag and candidate[1] > bottom:
      break
  return len(sand) - initial - (1 if not flag else 0)

day14(*data14), day14(*data14, True)

(1061, 25055)

# Day 15

In [151]:
def parse15(data):
  sensors, beacons = [], []
  for line in data:
    match = re.match('Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)', line)
    sensors.append((int(match[1]), int(match[2])))
    beacons.append((int(match[3]), int(match[4])))
  return np.vstack(sensors), np.vstack(beacons)

data15 = parse15(data(15))

In [153]:
def day15(sensors, beacons, target=2000000):
  tree = intervaltree.IntervalTree()
  for sensor, beacon in zip(sensors, beacons):
    distance = abs(sensor[0]-beacon[0]) + abs(sensor[1] - beacon[1])
    for i in range(distance+1):
      if sensor[1] + i == target or sensor[1] - i == target:
        tree.addi(sensor[0] - distance + i, sensor[0] + distance - i + 1)
  return tree

sensor_ranges = day15(*data15, 10)

In [157]:
sensor_ranges.merge_overlaps()
sensor_ranges

IntervalTree([Interval(-2, 25)])

In [145]:
sensor_ranges[10].merge_overlaps()
sensor_ranges[10].end() - sensor_ranges[10].begin() - 1

26

In [92]:
import intervaltree
t = intervaltree.IntervalTree()
t[10:11] = 2
t[4:6] = 1
t[0:5] = 1
t.merge_overlaps()

In [90]:
t

IntervalTree([Interval(0, 6), Interval(10, 11, 2)])

In [11]:
tau = 2*np.pi

rotation = np.array([[np.cos(tau/8), -np.sin(tau/8)],[np.sin(tau/8), np.cos(tau/8)]])
undo = np.array([[np.cos(-tau/8), -np.sin(-tau/8)],[np.sin(-tau/8), np.cos(-tau/8)]])

sensors, beacons = (rotation @ data15[0]).transpose(), (rotation @ data15[1]).transpose()


In [14]:
sensors, beacons

(array([[-11.3137085 ,  14.14213562],
        [ -4.94974747,  17.67766953],
        [  7.77817459,  10.60660172],
        [ -1.41421356,  18.38477631],
        [ -7.07106781,  21.21320344],
        [ -2.12132034,  21.92031022],
        [  0.70710678,  10.60660172],
        [  1.41421356,   1.41421356],
        [ -7.77817459,   7.77817459],
        [  4.24264069,  24.04163056],
        [ -2.12132034,  26.1629509 ],
        [  6.36396103,  16.26345597],
        [  7.77817459,  12.02081528],
        [ 13.43502884,  14.8492424 ]]),
 array([[-12.02081528,   9.19238816],
        [ -4.24264069,  18.38477631],
        [  8.48528137,  12.72792206],
        [ -4.24264069,  18.38477631],
        [ -4.24264069,  18.38477631],
        [ -4.24264069,  18.38477631],
        [ -5.65685425,   8.48528137],
        [ -5.65685425,   8.48528137],
        [ -5.65685425,   8.48528137],
        [  5.65685425,  29.69848481],
        [ -0.70710678,  30.40559159],
        [  8.48528137,  12.72792206],
        [ 

In [65]:
def pairwise(iterable):
    # https://docs.python.org/dev/library/itertools.html#recipes
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)
    
# https://stackoverflow.com/questions/25068538/intersection-and-difference-of-two-rectangles
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
      self.x1 = min(x1, x2)
      self.x2 = max(x1, x2)
      self.y1 = min(y1, y2)
      self.y2 = max(y1, y2)

    def intersection(self, other):
        a, b = self, other
        x1 = max(min(a.x1, a.x2), min(b.x1, b.x2))
        y1 = max(min(a.y1, a.y2), min(b.y1, b.y2))
        x2 = min(max(a.x1, a.x2), max(b.x1, b.x2))
        y2 = min(max(a.y1, a.y2), max(b.y1, b.y2))
        if x1<x2 and y1<y2:
            return type(self)(x1, y1, x2, y2)
    __and__ = intersection

    def difference(self, other):
        inter = self&other
        if not inter:
            yield self
            return
        xs = {self.x1, self.x2}
        ys = {self.y1, self.y2}
        if self.x1<other.x1<self.x2: xs.add(other.x1)
        if self.x1<other.x2<self.x2: xs.add(other.x2)
        if self.y1<other.y1<self.y2: ys.add(other.y1)
        if self.y1<other.y2<self.y2: ys.add(other.y2)
        for (x1, x2), (y1, y2) in itertools.product(
            pairwise(sorted(xs)), pairwise(sorted(ys))
        ):
            rect = type(self)(x1, y1, x2, y2)
            if rect!=inter:
                yield rect
    __sub__ = difference

    def __init__(self, x1, y1, x2, y2):
        if x1>x2 or y1>y2:
            raise ValueError("Coordinates are invalid")
        self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2

    def __iter__(self):
        yield self.x1
        yield self.y1
        yield self.x2
        yield self.y2

    def area(self):
      return (self.y2 - self.y1) * (self.x2 - self.x1)

    def __eq__(self, other):
        return isinstance(other, Rectangle) and tuple(self)==tuple(other)
    def __ne__(self, other):
        return not (self==other)
    def __repr__(self):
        return type(self).__name__+repr(tuple(self))



In [104]:
rectangles = []
for i, sensor in enumerate(sensors):
  beacon = tuple(beacons[i])
  mirror = (2*sensor[0] - beacon[0], 2*sensor[1] - beacon[1])
  rectangles.append(Rectangle(
    min(mirror[0], beacon[0])-1, 
    min(mirror[1], beacon[1])-1, 
    max(mirror[0], beacon[0])+1, 
    max(mirror[1], beacon[1])+1
  ))
rectangles[6].area()

91.94112549695426

In [76]:
intersections = 0
for i in range(len(rectangles)):
  for j in range(i+1, len(rectangles)):
    if double := rectangles[i].intersection(rectangles[j]):
      intersections += double.area()

In [79]:
sum([rectangle.area() for rectangle in rectangles])

474.00000000000006

In [101]:
rectangles[6].area()

53.999999999999986

In [102]:
rectangles[6]

Rectangle(-5.65685424949238, 8.485281374238571, 7.071067811865476, 12.727922061357855)